In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from os.path import join, isfile
from os import listdir
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import string

In [2]:
def get_json(fname, fpath='../../wos_paper/wos_db_benchmark/benchmarking/'):
    fname = join(fpath, fname)
    if fname[-2:] == 'gz':
        with gzip.open(fname, "rb") as f:
            data = json.loads(f.read(), encoding="utf-8")
    else:
        with open(fname, 'r') as fp:
            data = json.load(fp)
    return data

def get_json_fnames(sourcepath='../../wos_paper/wos_db_benchmark/benchmarking/'):
    only_json = [f for f in listdir(sourcepath) if 
                     isfile(join(sourcepath, f)) 
                     and f[-4:] == 'json']
    return only_json


def get_result_fnames(sourcepath='../results/arango/'):
    r = [f for f in listdir(sourcepath) if 
                     isfile(join(sourcepath, f)) 
                     and f[-7:] == 'json.gz' and 'result' in f and 'limit' not in f]
    return r

def get_content(sql_list, fpath, how='sql'):
    report = []
    if how == 'sql':
        limit_tokens = 3
    elif how == 'arango':
        limit_tokens = 3
    else:
        limit_tokens = 3
    for f in sql_list:
        coded = f.split('_')
        limit = -1 if len(coded) < limit_tokens else int(coded[-1].split('.')[0])
        content = get_json(f, fpath)
        report += [(coded[0], limit, f, content)]
    return report

def get_report_gzipped(flist, fpath):
    acc = []
    for fname in flist:
        qname = fname.split('/')[0]
        data = get_json(fname, fpath)
        acc += [(qname, data)] 
    return acc

def get_mysql_times(sql_report):
    times_stat = []
    for item in sql_report:
        key, limit, f, content = item
        if 'elapsed' in content:
            times_stat += [(*item[:-1], content['elapsed'])]
        else:
            times_stat += [(*item[:-1], content['elapsed_1978'])]
    return times_stat

def get_arango_times(reports):
    times = [(*item[:-1], [sum(x.values()) for x in item[-1]]) for item in reports]
    times_stat = [(*item[:-1], np.mean(item[-1]), np.std(item[-1])) for item in times]
    return times_stat

In [3]:
sql_list = sorted(get_json_fnames())
sql_report = get_content(sql_list, '../../wos_paper/wos_db_benchmark/benchmarking/')

In [4]:
flist = sorted(get_result_fnames())
arango_data = get_report_gzipped(flist, '../results/arango/')

# Query 1

In [5]:
q1_sql = pd.DataFrame(sql_report[1][-1]['results_1978'], columns=['year', 'title', 'count'])
q1_arango = pd.DataFrame([{**item['journal'], **{'count': item['number_pubs']}} for item in arango_data[0][1]])

# q1_arango groupby by title for comparison
q1_arango_condensed = q1_arango.groupby('title')['count'].sum().reset_index().rename(columns={0:'count'})
q1_arango_condensed.sort_values('count', ascending=False).head()

,title,count
2836,FEDERATION PROCEEDINGS,7287
1528,BULLETIN OF THE AMERICAN PHYSICAL SOCIETY,5895
1973,CLINICAL RESEARCH,5208
11,ABSTRACTS OF PAPERS OF THE AMERICAN CHEMICAL S...,4514
1435,BRITISH MEDICAL JOURNAL,3597


In [6]:
q1_cmp = pd.merge(q1_sql, q1_arango, on='title', how='outer', suffixes=['_sql', '_arango'])
q1_cmp2 = pd.merge(q1_sql, q1_arango_condensed, on='title', how='outer', suffixes=['_sql', '_arango'])
q1_cmp2 = q1_cmp2.sort_values('count_arango', ascending=False)
q1_cmp2['cmp'] = (q1_cmp2['count_sql'] == q1_cmp2['count_arango'])
q1_cmp2[['title', 'count_sql', 'count_arango', 'cmp']].head(12)

,title,count_sql,count_arango,cmp
0,FEDERATION PROCEEDINGS,7287.0,7287,True
1,BULLETIN OF THE AMERICAN PHYSICAL SOCIETY,5895.0,5895,True
2,CLINICAL RESEARCH,5208.0,5208,True
3,ABSTRACTS OF PAPERS OF THE AMERICAN CHEMICAL S...,4514.0,4514,True
4,BRITISH MEDICAL JOURNAL,3597.0,3597,True
5,LANCET,3189.0,3189,True
6,TLS-THE TIMES LITERARY SUPPLEMENT,3062.0,3062,True
7,NATURE,2990.0,2990,True
8,TRANSACTIONS-AMERICAN GEOPHYSICAL UNION,2812.0,2812,True
9,DOKLADY AKADEMII NAUK SSSR,2339.0,2339,True


# Query 2

In [7]:
puncts = list(string.punctuation)
stop_words_nltk = set(stopwords.words('english'))
numerics = set([str(i) for i in range(100)])
all_stops = list(set(puncts) | stop_words_nltk | numerics)
# remove stopword from sql results
q2_sql = pd.DataFrame(sql_report[2][-1]['frequent_words'], columns=['uword', 'count'])
q2_sql_mod = q2_sql.loc[~q2_sql.uword.isin(all_stops)]

#remove '.2.' from arango results
q2_arango = pd.DataFrame(arango_data[1][1])
q2_arango = q2_arango.loc[~q2_arango.uword.apply(lambda x: x.startswith('.'))]


q2_cmp = pd.merge(q2_sql_mod, q2_arango, on='uword', how='outer', 
                  suffixes=['_sql', '_arango'])
# define error
q2_cmp['err'] = (q2_cmp['count_sql'] - q2_cmp['count_arango'])/q2_cmp['count_arango']

In [8]:
# define ranks
q2_cmp = q2_cmp.sort_values('count_sql', 
                            ascending=False).\
        reset_index().rename(columns={'index': '_'}).\
        reset_index().rename(columns={'index': 'rank_sql'})
q2_cmp = q2_cmp.sort_values('count_arango', 
                            ascending=False).\
        reset_index().rename(columns={'index': '_'}).\
        reset_index().rename(columns={'index': 'rank_arango'})
q2_cmp = q2_cmp.drop('_', axis=1)
q2_cmp.sort_values('count_sql', ascending=False).head(20)

,rank_arango,rank_sql,uword,count_sql,count_arango,err
0,0,0,effect,242666.0,242312.0,0.001461
1,1,1,study,189789.0,189016.0,0.004090
2,2,2,effects,185911.0,185526.0,0.002075
3,3,3,studies,185596.0,184634.0,0.005210
4,4,4,new,148332.0,147154.0,0.008005
5,5,5,analysis,114563.0,113725.0,0.007369
6,6,6,synthesis,107862.0,105913.0,0.018402
7,7,7,human,101709.0,101340.0,0.003641
10,10,8,structure,99601.0,97406.0,0.022535
8,8,9,properties,99019.0,98039.0,0.009996


# Query 3

In [9]:
sql_report[3][3]['results'][:]

[['SCHLEYER', 'PV', 16],
 ['SACTON', 'J', 14],
 ['MARICIC', 'S', 12],
 ['BALNER', 'H', 11],
 ['DAVIS', 'DH', 11],
 ['LUMBROSO', 'H', 11],
 ['MEBOLD', 'U', 11],
 ['MITCHISON', None, 11],
 ['MOLINA', 'R', 11],
 ['PERNIS', 'B', 11],
 ['ROBSON', 'EB', 11],
 ['WILSON', 'R', 11],
 ['AMOS', 'DB', 10],
 ['BAULIEU', 'EE', 10],
 ['BOKSENBERG', 'A', 10],
 ['CELADA', 'F', 10],
 ['CINADER', 'B', 10],
 ['CUNNINGHAM', 'B', 10],
 ['DICZFALUSY', 'E', 10],
 ['EDIDIN', 'M', 10],
 ['FOTHERBY', 'K', 10],
 ['FOX', 'L', 10],
 ['FUDENBERG', 'HH', 10],
 ['GALL', 'WE', 10],
 ['GOLDBERG', 'E', 10],
 ['GOODMAN', 'H', 10],
 ['GOSS', 'WM', 10],
 ['HARPER', 'MJK', 10],
 ['JONES', 'WR', 10],
 ['JURIC', 'M', 10],
 ['MILLER', 'JFAP', 10],
 ['MITCHELL', 'SF', 10],
 ['MONTWILL', 'A', 10],
 ['NOSSAL', 'GJV', 10],
 ['RYAN', 'KJ', 10],
 ['SCHMIDT', 'G', 10],
 ['SCHUMACHER', 'GFB', 10],
 ['SHIVERS', 'CA', 10],
 ['SOUTHAM', 'AL', 10],
 ['STEVENS', 'VC', 10],
 ['TALWAR', 'GP', 10],
 ['VAERMAN', 'JP', 10],
 ['VOISIN', 'GA', 10]

In [10]:
q3_sql = pd.DataFrame(sql_report[3][3]['results'][:], 
                      columns=['last_name', 'first_name', 'cnt']).sort_values(['cnt', 'last_name', 'first_name'], 

SyntaxError: unexpected EOF while parsing (<ipython-input-10-b7bf38982fd1>, line 2)

In [ ]:
q3_sql.head()

In [ ]:
q3_sql = pd.DataFrame(sql_report[3][3]['results'][:], 
                      columns=['last_name', 'first_name', 'cnt']).sort_values(['cnt', 'last_name', 'first_name'], 
                                                                              ascending=False)
q3_sql = q3_sql[['last_name', 'first_name', 'cnt']]

q3_arango = pd.DataFrame(arango_data[2][1]).sort_values(['cnt', 'last_name', 'first_name'], ascending=False)
q3_arango = q3_arango.loc[(q3_arango.last_name != '') & (q3_arango.first_name != '')]
q3_arango = q3_arango[['last_name', 'first_name', 'cnt']]

In [ ]:
q3_cmp = pd.merge(q3_sql, q3_arango, on=['last_name', 'first_name'], suffixes=['_sql', '_arango'])

In [ ]:
q3_cmp.head(20)

# Query 4

In [ ]:
q4_sql = pd.DataFrame(sql_report[4][-1]['results']).rename(columns={'A': 'wosid'})
q4_arango = pd.DataFrame([{'ratio': dd['f'], 'wosid': w} for dd in arango_data[3][-1] for w in dd['ids']])
q4_cmp = pd.merge(q4_sql[['wosid', 'ratio']], q4_arango, on='wosid', how='left', suffixes=['_sql', '_arango'])
q4_cmp['err'] = abs((q4_cmp['ratio_sql'] - q4_cmp['ratio_arango'])/q4_cmp['ratio_arango'])
q4_cmp['flag'] = (q4_cmp['err'] > 1e-3)
q4_cmp.head(20)

In [ ]:
sum(q4_cmp.flag)

In [ ]:
q4_cmp.loc[q4_cmp.flag]

# Query 5 : only arango ready

In [ ]:
ww = arango_data[4][1]

In [ ]:
jas = sorted([x['ja'] for x in ww])
jbs = set([y['jb'] for x in ww for y in x['stats']])
jall = sorted(set(jas) | jbs)
print(len(jas), len(jbs), len(jall))

In [ ]:
acc = []
jas = [item['ja'] for item in ww]
for item in ww[:]:
    arr = [x['s'] for x in item['stats']]
    ixs = [x['jb'] for x in item['stats']]
    acc += [pd.Series(arr, index=ixs)]
q5_arango = pd.concat(acc, axis=1, sort=True)
q5_arango = q5_arango.rename(columns=dict(zip(range(len(jas)), jas)))

In [ ]:
q5_arango.head()

In [ ]:
q5_arango.shape

In [ ]:
sql_report[5][-1]['results'][:3]

In [ ]:
#q6
arango_data[-1]